# Ejemplos de optimización por enjambres de partículas (PSO)
---

Tutorial completo: https://deap.readthedocs.io/en/master/examples/pso_basic.html

Referencia DEAP: http://deap.readthedocs.io/en/master/api/tools.html



In [2]:
import operator
import random

import numpy
import math

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if abs(speed) < part.smin:
            part.speed[i] = math.copysign(part.smin, speed)
        elif abs(speed) > part.smax:
            part.speed[i] = math.copysign(part.smax, speed)
    part[:] = list(map(operator.add, part, part.speed))

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)

    return pop, logbook, best


p, l, b = main()

gen	evals	avg      	std      	min      	max     
0  	5    	0.0900617	0.0600924	0.0315672	0.197888
1  	5    	0.0781385	0.0530942	0.00231065	0.166234
2  	5    	0.142458 	0.0340245	0.101318  	0.190903
3  	5    	0.135292 	0.0744631	0.0234854 	0.230706
4  	5    	0.267417 	0.221149 	0.0502567 	0.679538
5  	5    	0.274372 	0.130055 	0.0716898 	0.433902
6  	5    	0.26129  	0.10606  	0.132719  	0.421042
7  	5    	0.224717 	0.126744 	0.0131532 	0.39977 
8  	5    	0.301819 	0.344363 	0.0347529 	0.974959
9  	5    	0.164408 	0.15644  	0.0148248 	0.434948
10 	5    	0.417956 	0.493899 	0.0441076 	1.38355 
11 	5    	0.321857 	0.174626 	0.0486038 	0.520259
12 	5    	0.34031  	0.113319 	0.194368  	0.47246 
13 	5    	0.849858 	0.586186 	0.15923   	1.58445 
14 	5    	0.32105  	0.115047 	0.187161  	0.476872
15 	5    	0.317286 	0.147389 	0.0848849 	0.460117
16 	5    	0.636916 	0.587647 	0.175908  	1.79512 
17 	5    	0.34215  	0.0662831	0.22342   	0.398686
18 	5    	0.704943 	0.628727 	0.194153  	1.9189  
19

554	5    	0.334327 	0.161744 	0.170155   	0.560625
555	5    	0.262404 	0.146103 	0.0100976  	0.445156
556	5    	0.370251 	0.219618 	0.113781   	0.733065
557	5    	0.289235 	0.155474 	0.110143   	0.471823
558	5    	0.574819 	0.232003 	0.137633   	0.81979 
559	5    	0.270869 	0.0449513	0.200836   	0.34031 
560	5    	0.433306 	0.407728 	0.130583   	1.22794 
561	5    	0.243029 	0.112367 	0.0786146  	0.39738 
562	5    	0.535267 	0.292316 	0.284856   	1.10868 
563	5    	0.426927 	0.170178 	0.214682   	0.70577 
564	5    	0.639181 	0.449853 	0.283595   	1.52128 
565	5    	0.446278 	0.270186 	0.0800558  	0.871613
566	5    	0.340061 	0.100917 	0.20704    	0.472919
567	5    	0.457401 	0.396217 	0.15513    	1.24052 
568	5    	0.518332 	0.241936 	0.223929   	0.900323
569	5    	0.461435 	0.43033  	0.11566    	1.30075 
570	5    	0.361227 	0.0812195	0.21477    	0.447939
571	5    	0.504262 	0.411569 	0.215925   	1.31601 
572	5    	0.363251 	0.0301976	0.315392   	0.408821
573	5    	0.482052 	0.333385 	0

865	5    	0.28065  	0.141067 	0.0728001  	0.45195 
866	5    	0.423808 	0.186155 	0.173297   	0.680042
867	5    	0.330951 	0.235511 	0.114963   	0.784734
868	5    	0.350361 	0.104594 	0.206775   	0.475169
869	5    	0.404943 	0.267642 	0.180521   	0.917385
870	5    	0.512771 	0.502146 	0.0202364  	1.46472 
871	5    	0.357747 	0.107021 	0.179938   	0.491925
872	5    	0.381691 	0.431378 	0.0238892  	1.21765 
873	5    	0.508212 	0.4188   	0.160916   	1.31424 
874	5    	0.396722 	0.26901  	0.052663   	0.745369
875	5    	0.389577 	0.457957 	0.0419671  	1.28771 
876	5    	0.438684 	0.276661 	0.160282   	0.946175
877	5    	0.228718 	0.112922 	0.110314   	0.371806
878	5    	0.837733 	0.72693  	0.0586611  	1.93228 
879	5    	0.260726 	0.0925087	0.116051   	0.369102
880	5    	0.488951 	0.290965 	0.162182   	1.00189 
881	5    	0.234067 	0.0983953	0.100488   	0.397872
882	5    	0.588714 	0.39845  	0.314746   	1.37888 
883	5    	0.31909  	0.0551043	0.24598    	0.406231
884	5    	0.363046 	0.108001 	0

In [3]:
b

[8.688399417537477, 6.784013253478751]